In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

tokenizer = AutoTokenizer.from_pretrained("kreimben/CodeMind-gemma", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("kreimben/CodeMind-gemma")

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--kreimben--CodeMind-gemma. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [2]:
model = model.to('cuda')

In [9]:
%%time

res = get_completion(
    'i dont know about word break problem. please give me an approch about this problem in python.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    # device='cpu',
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


CPU times: total: 13.9 s
Wall time: 25.3 s


In [10]:
answer = res.split('model')[-1]

In [11]:
from IPython.display import display, Markdown

answer = answer.replace('\\\\', '\\')

display(Markdown(answer))

 Given a dictionary and a word, find all rhymes. You can choose any letter to find the rhyme for it. Two words are considered close if the absolute difference in their lexicographical sum is less than or equal to 3. Analyze the word break problem in Python. We have to first get the word and the dictionary. Then we will calculate the total word score by iterating over the words in the dictionary and summing them up.  For each word, we will split it into the individual characters for better analysis of the lexicographical sum. The algorithm will be similar to this. The algorithm will be similar to this. 
Let's see the implementation. 
```python
def wordBreak(words, chars):
    dp = [False] * len(words)

    # initial state 0-indexed
    dp[0] = True

    # now iterate over and add edges to each dp cell
    for i in range(1, len(words)):
        for j in range(i):
            if words[i] == words[j][-1] and j+1 in dp:
                dp[i] = True

    # find the path from 0 to len-1
    # find the path from 0 to len-1
    ansp = ""
    k = len(words) - 1
    while k >= 0:
        if not dp[k]:
            break
        ansp += words[k]
        k -= 1

    return ansp[::-1]
```
This solution runs in O(n * m) time and uses O(n) additional space.
Let's observe the steps, we find the optimal solution to the above problem, when the given string s is the empty string:  What about if the given string is composed of exactly one word in the dictionary, should the optimal solution be the same as the simple version of the above problem?  Certainly!  The key to solve this problem is to split the given word in such a way that we will just take the last word or simply the last character of the split word.  For the first split of the word, it is obvious that we have to take the last word as that would be the word.  Suppose we are given this word in "word Break" 6 characters:  Now the character at index 0 of the word, which is "B " and the last character of it, which is "E ", are not matched and we take "W " in the first split, since they are not the same, so the problem is not clear, now for first 2 characters "W " and "F " at index 0 and 2 of the "W " are not matched and we take "E " in the first split, we take "W " in the second split, now it is clear.  The third problem is that if we use the character "W " and "E " then the last character of the "W " would be not matched but in the first and second splits, the last characters match with "W ".  Therefore,  We can observe that in this problem the same sub strings take only one more position at index 0, we take the whole thing for our first split and leave the position at index 0 for next iteration, as we are going to do the two remaining sub strings for that, the character at the last index of the string, namely that "E ", "D ", and "B " would be not matched so we take the corresponding characters and leave the position for next iteration, let's take the "D " and we take "B ", for the problem of 3 character we are keeping this character, for the next iteration, now, for the first character of the second split, we will take the "B " so that would be "B " and "E ", we leave the position before that to next iteration but now for the "I ", we will take "L ", so, for the position 2, we will take "B ", before that, we will take "L ", which means that we will take them both in our first iteration.  The third sub word from sub word of B and E, we take 1 more position "D " so for our first two sub words, we take them "B ", "D ", "E ", now our two splits of the string are such  1. B D E 2. B D
Now, let's keep analyzing all the elements in the given word, and our goal is that both elements for first split and our chosen character (we use "L " and "W ") have to be matched and not the characters, we take them all and now, how we will do the 3rd split, we take "L " and "E ", then the position before that to next iteration is not taken so we will take the "D " and